In [ ]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import holodeck as holo
import holodeck.single_sources as ss
import holodeck.cyutils as cy
import matplotlib.pyplot as plt
import kalepy as kale


import matplotlib.cm as cm
from holodeck import plot, utils
from holodeck.constants import YR, MSOL, GYR

# Check flag replacement
## Fixed_Time

In [ ]:
dur, cad = 17.5*YR, 0.2*YR
fobs = utils.nyquist_freqs(dur,cad)
fobs_edges = utils.nyquist_freqs_edges(dur,cad)
sam = holo.sam.Semi_Analytic_Model()
# sam = holo.sam.Semi_Analytic_Model(mtot=(1.0e4*MSOL, 1.0e11*MSOL, 20), mrat=(1e-3, 1.0, 20), redz=(1e-3, 10.0, 20))  # faster version
# hard = holo.hardening.Fixed_Time.from_sam(sam, 3*GYR)
# fobs_orb_edges = fobs_edges / 2.0 
# fobs_orb_cents = fobs/ 2.0
# edges, dnum = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents) #, zero_stalled=True) # should the zero stalled option be part of the parameter space?
# edges[-1] = fobs_orb_edges


In [ ]:
# just using gwb() function
gwb = sam.gwb(fobs_edges, realize=10)
fig = plot.plot_gwb(fobs, gwb)

In [ ]:
# just using gwb() function

gwb = sam.gwb(fobs_edges, hard=holo.hardening.Hard_GW, realize=10)
fig = plot.plot_gwb(fobs, gwb)

Fixed_Time dadt issue: Need to create a Fixed_Time instance, like by hard = holo.hardening.Fixed_Time.from_sam(sam, 3*GYR)

# ss_gws_redz using sam.ss_gwb()
## with Hard_GW

In [ ]:
hc_ss, hc_bg, sspar, bgpar = sam.ss_gwb(fobs_edges, hard=holo.hardening.Hard_GW, realize = 10, loudest = 5, params = True)

In [ ]:
fig = plot.plot_bg_ss(fobs, hc_bg, hc_ss)

In [ ]:
fig = plot.plot_pars(fobs, hc_ss, hc_bg, sspar, bgpar)

In [ ]:
hc_ss, hc_bg = sam.ss_gwb(fobs_edges, realize=30, loudest=100, params=False)
fig = plot.plot_bg_ss(fobs, hc_bg, hc_ss)

## With Fixed_Time

In [ ]:
hard = holo.hardening.Fixed_Time.from_sam(sam, 3*GYR)
hc_ss, hc_bg, sspar, bgpar = sam.ss_gwb(fobs_edges, hard=hard, realize = 10, loudest = 5, params = True)
fig = plot.plot_bg_ss(fobs, hc_bg, hc_ss)
fig = plot.plot_pars(fobs, hc_ss, hc_bg, sspar, bgpar )

In [ ]:
hc_ss, hc_bg = sam.ss_gwb(fobs_edges, hard=hard, realize = 10, loudest = 5, params = False)
fig = plot.plot_bg_ss(fobs, hc_bg, hc_ss)

# Change redz for params

In [ ]:
dur, cad = 16*YR, 0.2*YR
fobs_gw_cents = utils.nyquist_freqs(dur,cad)
fobs_gw_edges = utils.nyquist_freqs_edges(dur,cad)
# sam = holo.sam.Semi_Analytic_Model(mtot=(1e5*MSOL, 1e12*MSOL, 20), mrat = (0.001, 1, 20), redz=(0.001, 10, 20))
sam=holo.sam.Semi_Analytic_Model()

# hard = holo.hardening.Fixed_Time.from_sam(sam, 3*GYR)
hard = holo.hardening.Hard_GW
# hard = holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR)

fobs_orb_cents = fobs_gw_cents / 2.0
# edges, dnum, redz_final = sam._dynamic_binary_number_at_fobs_consistent(hard, fobs_orb_cents)
edges, dnum, redz_final = sam._dynamic_binary_number_at_fobs_inconsistent(hard, fobs_orb_cents)

In [ ]:
print(redz_final.shape)

In [ ]:

hc_ss, hc_bg, sspar, bgpar = sam.ss_gwb(fobs_gw_edges, hard, params=True)

In [ ]:
fobs_orb_edges = fobs_gw_edges / 2.0
fobs_orb_cents = fobs_gw_cents / 2.0

edges, dnum, redz_final = sam.dynamic_binary_number_at_fobs(hard, fobs_orb_cents)
edges[-1] = fobs_orb_edges

number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
number = number * np.diff(np.log(fobs_gw_edges))
print(f"number: {utils.stats(number)}")
print(f"number.sum(): {number.sum():.4e}")

In [ ]:
use_redz = sam._redz_final